In [1]:
# 網頁透過Flask-Alchemy連接資料庫
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy

# create the extension
db = SQLAlchemy()
# create the app
app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI'] = 'mssql+pyodbc:///?' \
'driver=ODBC+Driver+17+for+SQL+Server&' \
'trusted_connection=yes&' \
'server=LAPTOP-E523KF0A&' \
'database=testdb1'

# initialize the app with the extension
db.init_app(app)

class score(db.Model):

    __tablename__='score' 
    
    編號 = db.Column(db.Integer, primary_key = True)
    學校 = db.Column(db.String(200))
    公告名額112 = db.Column(db.Integer)
    公告名額113 = db.Column(db.Integer)
    差額 = db.Column(db.Integer)
    累進推估名次113 = db.Column(db.String(200))
    回報分數112 = db.Column(db.DECIMAL(4, 1))
    預估分數113 = db.Column(db.DECIMAL(4, 1))
      
    
    def __init__(self, 學校, 公告名額112, 公告名額113, 差額, 累進推估名次113, 回報分數112, 預估分數113):
       self.學校 = 學校
       self.公告名額112 = 公告名額112
       self.公告名額113 = 公告名額113
       self.差額 = 差額
       self.累進推估名次113 = 累進推估名次113
       self.回報分數112 = 回報分數112
       self.預估分數113 = 預估分數113
       
# 以上類別建好囉
with app.app_context():
    # db.drop_all()   #如果資料表已存在就刪除
    db.create_all() # #新增資料表
print('資料庫連線成功！')

import pandas as pd

def import_data():
    # Read the Excel file
    file_path = '落點.xlsx'
    df = pd.read_excel(file_path)
    
    # Iterate over the rows in the DataFrame and add them to the database
    for _, row in df.iterrows():
        score_record = score(
            學校=row['學校'],
            公告名額112=row['112公告名額'],
            公告名額113=row['113公告名額'],
            差額=row['差額'],
            累進推估名次113=row['113累進推估名次'],
            回報分數112=row['112回報分數'],
            預估分數113=row['113預估分數']
        )
        db.session.add(score_record)
    db.session.commit()

@app.route("/import")
def import_view():
    import_data()
    return "Data imported successfully!"


@app.route("/scores")
def scores_view():
    scores = score.query.limit(57).all()
    return render_template("scores.html", scores=scores)

app.run()

資料庫連線成功！
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [30]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import pandas as pd

# 启动dash
app = dash.Dash()

# 载入数据集
file_path = '落點.xlsx'
sales_data = pd.read_excel(file_path)

# 确保所有学校名称都显示在横轴上
tickvals = list(range(len(sales_data['學校'])))
ticktext = sales_data['學校']

# 设置网页界面
app.layout = html.Div([
    dcc.Graph(
        id='First Dash Graph',
        # Plotly的 Figure是由data与graph构成的
        figure={
            # 绘图
            'data': [
                go.Scatter(
                    x=tickvals,
                    y=sales_data['112公告名額'],
                    mode='lines+markers',
                    name='112公告名額'
                ),
                go.Scatter(
                    x=tickvals,
                    y=sales_data['113公告名額'],
                    mode='lines+markers',
                    name='113公告名額'
                )
            ],
            # 设置图形界面
            'layout': go.Layout(
                title="112公告名額 vs 113公告名額",
                xaxis={
                    'title': {'text': '學校', 'standoff': 20},
                    'tickmode': 'array',
                    'tickvals': tickvals,
                    'ticktext': ticktext,
                    'tickangle': -70  # 使学校名称垂直显示
                },
                yaxis={'title': {'text': "名額", 'standoff': 20}},
                hovermode='closest',
                margin={'t': 40, 'b': 150, 'l': 80, 'r': 40},  # 调整图表的边距
                height=400  # 增加图表的高度
            )
        }
    )
])

if __name__ == '__main__':
    app.run_server(port=5000, debug=True)




In [23]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import pandas as pd

# 启动dash
app = dash.Dash()

# 载入数据集
file_path = '落點.xlsx'
sales_data = pd.read_excel(file_path)

# 确保所有学校名称都显示在横轴上
tickvals = list(range(len(sales_data['學校'])))
ticktext = sales_data['學校']

# 设置网页界面
app.layout = html.Div([
    dcc.Graph(
        id='First Dash Graph',
        # Plotly的 Figure是由data与graph构成的
        figure={
            # 绘图
            'data': [
                go.Scatter(
                    x=tickvals,
                    y=sales_data['112公告名額'],
                    mode='lines+markers',
                    name='112公告名額'
                ),
                go.Scatter(
                    x=tickvals,
                    y=sales_data['113公告名額'],
                    mode='lines+markers',
                    name='113公告名額'
                )
            ],
            # 设置图形界面
            'layout': go.Layout(
                title="112公告名額 vs 113公告名額",
                xaxis={
                    'title': '學校',
                    'tickmode': 'array',
                    'tickvals': tickvals,
                    'ticktext': ticktext,
                    'tickangle': -90  # 使学校名称垂直显示
                },
                yaxis={'title': "名額"},
                hovermode='closest',
                margin={'t': 40, 'b': 150, 'l': 40, 'r': 40},  # 调整图表的边距
                height=600  # 增加图表的高度
            )
        }
    )
])

if __name__ == '__main__':
    app.run_server(port=5000, debug=True)

